# Домашняя работа №2

## 0. Импорты

In [1]:
from os import listdir
import pandas as pd
from nltk.corpus import stopwords
from spacy import load
from rank_bm25 import BM25Okapi
from gensim.models import KeyedVectors
import numpy as np
from navec import Navec
from numpy.linalg import norm
from sys import getsizeof
from nltk import sent_tokenize
from transformers import AutoTokenizer, AutoModel
from torch import no_grad
import pickle

C:\Users\robib\anaconda3\lib\site-packages\thinc\compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
C:\Users\robib\anaconda3\lib\site-packages\thinc\compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


## 1. Корпус

Я взял корпус новостей Collection5, создававшийся для NER. В нём собрана 1000 достаточно коротких текстов в виде .txt-файлов (а также аннотации, не нужные мне). Этот корпус позволил избежать необходимости разбираться с лишними данными и ускорить работу. Источник: https://github.com/natasha/corus#reference, а именно данные по http://www.labinform.ru/pub/named_entities/collection5.zip

In [2]:
d = {'text': []}

for file in listdir('collection5'):
    if str(file).endswith('txt'):
        with open(f'collection5/{str(file)}', encoding='utf-8') as f:
            d['text'].append(f.read())

df = pd.DataFrame(data=d)
df.head()

,text
0,Россия рассчитывает на конструктивное воздейст...
1,Комиссар СЕ критикует ограничительную политику...
2,"Пулеметы, автоматы и снайперские винтовки изъя..."
3,4 октября назначены очередные выборы Верховног...
4,Следственное управление при прокуратуре требуе...


In [3]:
with open("corpus.pickle", "wb") as f:
    pickle.dump(df, f)

## 2. Предобработка
Удаляются служебные части речи, стоп-слова, небуквенные токены. PoS-тэги через Spacy

In [4]:
stops = stopwords.words('russian')
nlp = load("ru_core_news_lg")

C:\Users\robib\anaconda3\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'ru_core_news_lg' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
def preprocess_text(text: str, pos_needed=False) -> str:
    doc = nlp(text)
    
    text_lemmas = []
    if pos_needed is False:
        for sent in doc.sents:
            for i in sent: 
                if i.text.isalpha() is True \
                and i.text not in stopwords.words('russian') \
                and i.pos_ in ['ADJ', 'ADV', 'NOUN', 'PROPN', 'VERB']:
                    text_lemmas.append(i.lemma_)
    elif pos_needed is True:
        for sent in doc.sents:
            for i in sent: 
                if i.text.isalpha() is True \
                and i.text not in stopwords.words('russian') \
                and i.pos_ in ['ADJ', 'ADV', 'NOUN', 'PROPN', 'VERB']:
                    text_lemmas.append(i.lemma_ + '_' + i.pos_)
    
    return text_lemmas

In [6]:
df['lemmas_list'] = df.text.apply(preprocess_text)
df['lemmas_pos_list'] = df.text.apply(preprocess_text, pos_needed=True)
df.head()

,text,lemmas_list,lemmas_pos_list
0,Россия рассчитывает на конструктивное воздейст...,"[россия, рассчитывать, конструктивный, воздейс...","[россия_PROPN, рассчитывать_VERB, конструктивн..."
1,Комиссар СЕ критикует ограничительную политику...,"[комиссар, се, критиковать, ограничительный, п...","[комиссар_NOUN, се_PROPN, критиковать_VERB, ог..."
2,"Пулеметы, автоматы и снайперские винтовки изъя...","[пулемёт, автомат, снайперский, винтовка, изъя...","[пулемёт_NOUN, автомат_NOUN, снайперский_ADJ, ..."
3,4 октября назначены очередные выборы Верховног...,"[октябрь, назначить, очередной, выбор, верховн...","[октябрь_NOUN, назначить_VERB, очередной_ADJ, ..."
4,Следственное управление при прокуратуре требуе...,"[следственный, управление, прокуратура, требов...","[следственный_ADJ, управление_NOUN, прокуратур..."


## 3. Обратный индекс BM-25Okapi через библиотеку


In [13]:
bm25 = BM25Okapi(df.lemmas_list)
bm25

In [28]:
with open("bm25.pickle", "wb") as f:
    pickle.dump(bm25, f)

## 4. Индекс текстов через word2vec
Я загрузил модель ruwikiruscorpora_upos_cbow_300_10_2021 с сайта RusVectōrēs https://rusvectores.org/ru/models/. Вектор документа находится как среднее арифметическое векторов входящих в него слов. Слова, которых нет в модели, выкидываются из суммы

In [14]:
def delete_OOVs(lemmas_list, model):
    clean = []
    for lemma in lemmas_list:
        try:
            model[lemma]
            clean.append(lemma)
        except KeyError:
            pass
    return clean

In [15]:
w2v_model = KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [16]:
df['lemmas_pos_list_w2v'] = df.lemmas_pos_list.apply(delete_OOVs, model=w2v_model)
df.head()

,text,lemmas_list,lemmas_pos_list,lemmas_pos_list_w2v
0,Россия рассчитывает на конструктивное воздейст...,"[россия, рассчитывать, конструктивный, воздейс...","[россия_PROPN, рассчитывать_VERB, конструктивн...","[россия_PROPN, рассчитывать_VERB, конструктивн..."
1,Комиссар СЕ критикует ограничительную политику...,"[комиссар, се, критиковать, ограничительный, п...","[комиссар_NOUN, се_PROPN, критиковать_VERB, ог...","[комиссар_NOUN, се_PROPN, критиковать_VERB, ог..."
2,"Пулеметы, автоматы и снайперские винтовки изъя...","[пулемёт, автомат, снайперский, винтовка, изъя...","[пулемёт_NOUN, автомат_NOUN, снайперский_ADJ, ...","[пулемёт_NOUN, автомат_NOUN, снайперский_ADJ, ..."
3,4 октября назначены очередные выборы Верховног...,"[октябрь, назначить, очередной, выбор, верховн...","[октябрь_NOUN, назначить_VERB, очередной_ADJ, ...","[октябрь_NOUN, назначить_VERB, очередной_ADJ, ..."
4,Следственное управление при прокуратуре требуе...,"[следственный, управление, прокуратура, требов...","[следственный_ADJ, управление_NOUN, прокуратур...","[следственный_ADJ, управление_NOUN, прокуратур..."


In [17]:
w2v_index = np.zeros((1000, 300)) # 1000 texts, vectors of length 300
for n, lemmas_list in enumerate(df['lemmas_pos_list_w2v']):
    w2v_index[n] = np.mean(w2v_model[lemmas_list], axis=0)
w2v_index.shape

(1000, 300)

In [18]:
with open("w2v_index.pickle", "wb") as f:
    pickle.dump(w2v_index, f)

## 5. Индекс текстов через navec
Я использовал модель Навек для новостных текстов с сайта https://github.com/natasha/navec. (Снова) вектор документа находится как среднее арифметическое векторов входящих в него слов, а слова, которых нет в модели, выкидываются из суммы

In [19]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')

In [20]:
df['lemmas_list_navec'] = df.lemmas_list.apply(delete_OOVs, model=navec)
df.head()

,text,lemmas_list,lemmas_pos_list,lemmas_pos_list_w2v,lemmas_list_navec
0,Россия рассчитывает на конструктивное воздейст...,"[россия, рассчитывать, конструктивный, воздейс...","[россия_PROPN, рассчитывать_VERB, конструктивн...","[россия_PROPN, рассчитывать_VERB, конструктивн...","[россия, рассчитывать, конструктивный, воздейс..."
1,Комиссар СЕ критикует ограничительную политику...,"[комиссар, се, критиковать, ограничительный, п...","[комиссар_NOUN, се_PROPN, критиковать_VERB, ог...","[комиссар_NOUN, се_PROPN, критиковать_VERB, ог...","[комиссар, се, критиковать, ограничительный, п..."
2,"Пулеметы, автоматы и снайперские винтовки изъя...","[пулемёт, автомат, снайперский, винтовка, изъя...","[пулемёт_NOUN, автомат_NOUN, снайперский_ADJ, ...","[пулемёт_NOUN, автомат_NOUN, снайперский_ADJ, ...","[пулемёт, автомат, снайперский, винтовка, изъя..."
3,4 октября назначены очередные выборы Верховног...,"[октябрь, назначить, очередной, выбор, верховн...","[октябрь_NOUN, назначить_VERB, очередной_ADJ, ...","[октябрь_NOUN, назначить_VERB, очередной_ADJ, ...","[октябрь, назначить, очередной, выбор, верховн..."
4,Следственное управление при прокуратуре требуе...,"[следственный, управление, прокуратура, требов...","[следственный_ADJ, управление_NOUN, прокуратур...","[следственный_ADJ, управление_NOUN, прокуратур...","[следственный, управление, прокуратура, требов..."


In [21]:
navec_index = np.zeros((1000, 300)) # 1000 texts, vectors of length 300
for n, lemmas_list in enumerate(df['lemmas_list_navec']):
    vec = np.zeros((1, 300))
    for word in lemmas_list:
        vec += navec[word]
    navec_index[n] = vec / len(lemmas_list)
navec_index.shape

(1000, 300)

In [22]:
with open("navec_index.pickle", "wb") as f:
    pickle.dump(navec_index, f)

## 6. Индекс с помощью BERT
По каждому документу проходимся по предложениям, считаем эмбеддинг для предложения (берём CLS, хотя разработчики модели рекомендуют среднее арифметическое токенов), складываем все эти эмбеддинги и делим на количество предложений в документе - так получаем эмбеддинг документа

In [24]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
BERT_model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [25]:
def encode(sent, tokenizer, model):
        for s in [sent]:
            encoded_input = tokenizer(s, padding=True, truncation=True, max_length=64, return_tensors='pt')
            with no_grad():
                model_output = model(**encoded_input)
        
        return model_output[0][0][0].numpy()

In [26]:
len(encode(df['text'][0], tokenizer, BERT_model))

1024

In [27]:
bert_index = np.zeros((1000, 1024)) # 1000 texts, vectors of length 1024
for n, text in enumerate(df['text']):
    vec = np.zeros((1, 1024))
    text_list = sent_tokenize(text)
    for sent in text_list:
        vec += encode(sent, tokenizer, BERT_model)
    bert_index[n] = vec / len(text_list)
bert_index.shape

(1000, 1024)

In [36]:
with open("bert_index.pickle", "wb") as f:
    pickle.dump(bert_index, f)

## 7. Поиск
В качестве оценки сходства используется косинусная близость (кроме BM-25, где своё ранжирование). Я показываю 2 самых релевантных документа, но имеется параметр, чтобы управлять этим числом. Вектор запроса находится как среднее арифметическое векторов входящих в него слов, а слова, которых нет в модели, выкидываются из суммы

In [34]:
def search_index(query: str, index_type: str, n=2):
    
    if index_type == 'bm25':
        
        clean_query = preprocess_text(query)
        
        ranked_docs = bm25.get_top_n(clean_query, df.text, n=n)
    
        return ranked_docs
    
    elif index_type == 'w2v':
        
        clean_query = delete_OOVs(preprocess_text(query, pos_needed=True), model=w2v_model)
        
        query_vector = np.mean(w2v_model[clean_query], axis=0)
        
        cos_sims = np.dot(w2v_index, query_vector) / (norm(w2v_index, axis=1) * norm(query_vector))
        
        d = {}
        for i, j in enumerate(cos_sims):
            d[i] = j
        ranking = sorted(d.items(), key=lambda x: x[1], reverse=True)
        
        return [df['text'][ranking[i][0]] for i in range(n)]
        
    elif index_type == 'navec':
        
        clean_query = delete_OOVs(preprocess_text(query), model=navec)
        
        vec = np.zeros((1, 300))
        for word in clean_query:
            vec += navec[word]
        query_vector = vec / len(clean_query)
        
        cos_sims = np.dot(navec_index, query_vector.transpose()) / (norm(navec_index, axis=1) * norm(query_vector))
        
        d = {}
        for i, j in enumerate(cos_sims):
            d[i] = j[0]
        
        ranking = sorted(d.items(), key=lambda x: x[1], reverse=True)
        
        return [df['text'][ranking[i][0]] for i in range(n)]
    
    elif index_type == 'bert':
        
        vec = np.zeros((1, 1024))
        query_list = sent_tokenize(query)
        for sent in text_list:
            vec += encode(sent, tokenizer, BERT_model)
        query_vector = vec / len(text_list)
        
        cos_sims = np.dot(bert_index, query_vector.transpose()) / \
            (norm(bert_index, axis=1) * norm(query_vector))
        
        d = {}
        for i, j in enumerate(cos_sims):
            d[i] = j[0]
        
        ranking = sorted(d.items(), key=lambda x: x[1], reverse=True)

        return [df['text'][ranking[i][0]] for i in range(n)]
    
    else:
        
        print('Invalid index type')

## 8. Сравнение реузльтатов
Время обработки (среднее из %%timeit) приведено для одного запроса, определённого выше. Нужно заметить, что создание индекса для w2v дольше за счёт необходимости парсить часть речи, и для BERT долгое, так как нейросеть энкодит

In [30]:
query = 'Телефон, оператор'

In [16]:
%%timeit
search_index(query, index_type='bm25')

10.4 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
search_index(query, index_type='w2v')

22.7 ms ± 3.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
search_index(query, index_type='navec')

23.3 ms ± 2.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
%%timeit
search_index(query, index_type='bert')

327 ms ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
print(getsizeof(bm25), getsizeof(w2v_index), getsizeof(navec_index), getsizeof(bert_index))

608164 2400128 2400128 8192128


| Index type    | Average time, ms  | Memory, bytes |
|:-------------:|:----------------- |:------------- |
| BM-25         | 10.4              | 608164        |
| Word2Vec      | 22.7              | 2400128       |
| Navec         | 23.3              | 2400128       |
| BERT          | 327               | 8192128       |

Замечание: результаты с BERT плохие и, видимо, зависят от длины запроса

In [37]:
search_index(query, index_type='bert')

['Вице-мэром Новосибирска по социальным вопросам стал учитель\n10:14 31.01.2013\n\nМэр Новосибирска Владимир Городецкий назначил своим заместителем по социальным вопросам бывшего замначальника городского управления образования Сергея Нелюбова; главным финансистом города стал бывший первый замглавы Бердска Владимир Штоп.',
 'Медведев назначил полпредом в ЦФО Олега Говоруна\n\nДмитрий Медведев назначил главу президентского управления внутренней политики Олега Говоруна полпредом главы государства в Центральном федеральном округе, сообщила во вторник пресс-служба Кремля.\n\nПост полпреда президента в ЦФО стал вакантным после того, как на прошлой неделе занимавший эту должность 11 лет Георгий Полтавченко стал губернатором Петербурга.']

## 9. Command Line Interface
Дольше из-за того, что при запуске команды в память загружается модель для индексирования запроса. Проблема: не парсятся запросы длиной больше одного слова

In [5]:
!python main.py телефон w2v 2

Execution time, s: 4.021940469741821

['Apple уволила менеджера, отвечавшего за картографический сервис\nApple logo\n© AFP/ Kimihiro Hoshino\nВ прошлом месяце Apple заявила об отставке главы по разработке iOS Скотта Форсталла, в ведение которого также входило создание картографического сервиса Apple.\n28/11/201211:37\nКлючевые слова: Apple\n\nНЬЮКАСЛ (Великобритания), 28 ноя — РИА Новости, Алина Гайнуллина. Компания Apple уволила менеджера Ричарда Уильямсона (Richard Williamson), руководившего командой по разработке ее картографического сервиса, и надеется вернуть доверие недовольных программой пользователей, сообщает агентство Bloomberg со ссылкой на информированные источники.\n\nМобильная операционная система iOS 6 стала первой, в которой в качестве штатного картографического приложения Apple использовала собственные карты, а не Google Maps. Новые карты Apple вызвали резкую критику пользователей за неточность отображения ландшафта и маршрутов, отсутствие маршрутов общественного транс

C:\Users\robib\anaconda3\lib\site-packages\thinc\compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
C:\Users\robib\anaconda3\lib\site-packages\thinc\compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]
C:\Users\robib\anaconda3\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'ru_core_news_lg' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
